<a href="https://colab.research.google.com/github/ydmarinb/spark/blob/main/02_performance_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performance tuning

El **performance tuning** en PySpark se refiere a optimizar y ajustar el rendimiento de las aplicaciones de PySpark para lograr un procesamiento más rápido y eficiente de los datos. Dado que PySpark opera en un entorno distribuido utilizando el framework de procesamiento distribuido Apache Spark, hay varios aspectos que se deben considerar para maximizar su rendimiento. Aquí tienes algunas estrategias y técnicas comunes para la optimización de rendimiento en PySpark:


**Particiones adecuadas:**
Las particiones son fragmentos de datos que se procesan en paralelo. Si hay un número insuficiente de particiones, no se aprovechará completamente el paralelismo. Si hay demasiadas, podría aumentar la sobrecarga de administración de particiones. Utiliza repartition para redistribuir los datos en un número deseado de particiones.

**Tamaño de la partición:**
El tamaño de las particiones es importante. Las particiones pequeñas pueden llevar a una sobrecarga debido a la administración excesiva de particiones, mientras que las particiones grandes pueden subutilizar recursos. Ajusta el tamaño según la memoria de los nodos y la naturaleza de los datos.

**Uso de caché:**
La caché es útil cuando planeas usar un DataFrame múltiples veces, evitando recomputaciones. Usa cache o persist con nivel de almacenamiento adecuado (MEMORY_ONLY, MEMORY_AND_DISK, etc.). Sin embargo, ten en cuenta que esto consume memoria.

**Broadcasting:**
Las tablas pequeñas pueden enviarse a cada nodo a través de broadcasting en lugar de redistribuir a través de la red. Utiliza broadcast para indicar que un DataFrame debe ser tratado como una tabla broadcast.

**Operaciones de transformación vs. acción:**
Spark realiza operaciones de transformación de manera perezosa, acumulando un plan de ejecución. Minimiza las acciones innecesarias para evitar la recomputación. Planea cuidadosamente las acciones para optimizar el flujo de trabajo.

**Uso eficiente de memoria:**
Configura correctamente la asignación de memoria para el driver y los ejecutores. Controla la proporción de memoria reservada para almacenamiento y ejecución. Ajusta spark.driver.memory, spark.executor.memory, spark.memory.fraction, etc.

**Configuración de recursos:**
Ajusta el número de nodos ejecutores, núcleos por ejecutor y memoria asignada según el tamaño del clúster y la naturaleza de la tarea. Asegúrate de no asignar demasiados recursos, lo que podría llevar a la competencia por recursos.

**Shuffling eficiente:**
Shuffling, la redistribución de datos entre particiones, puede ser costoso. Utiliza transformaciones como reduceByKey, aggregateByKey y combineByKey para reducir la necesidad de shuffling. Utiliza persist o cache después de una operación de shuffling para evitar recalculos.

**Formatos de almacenamiento:**
Utiliza formatos de almacenamiento eficientes como Parquet. Parquet es columnar y admite compresión, lo que mejora la velocidad de lectura y reduce el uso de almacenamiento.

**Optimización de consultas:**
Si utilizas Spark SQL, optimiza tus consultas. Utiliza EXPLAIN para entender el plan de ejecución. Utiliza índices o reorganiza tus datos si es posible para mejorar el rendimiento de las consultas.

**Monitoreo y ajuste:**
Utiliza las herramientas de monitoreo de Spark, como la interfaz web Spark UI, para rastrear el progreso de la aplicación, el uso de recursos, los tiempos de ejecución y más. Ajusta tu configuración y estrategias según lo que aprendas del monitoreo.



# Install pedendencies

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

# Define enviroment variables

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:

! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=617f6cf437541790b53991bbe0d0c9101a6261da1e0945f6c1c8860390e03e8a
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

base_dir = '/content/drive/MyDrive/retail_db'

# Catalys optimizer

<img src="https://www.databricks.com/wp-content/uploads/2018/05/Catalyst-Optimizer-diagram.png">

Para poder observar el diagrama anterior:

> Ir a la interface de apache -> SQL/DataFrame -> Seleccionar la operación de interes -> Details.

Wl objetivo en este caso es conocer el detalle de la ejecución de un proceso que esta llevando a cabo spark y asi poder optimizarlos.



In [ ]:
# Leyendo csv, y definiendo el tipo de datos y el nombre de columnas con el comando schema
df = spark.read.csv(f'{base_dir}/orders', schema='order_id INT, order_date DATE, order_customer_id INT, order_status STRING')


In [ ]:

from pyspark.sql.functions import count, col

df.\
  groupBy('order_status').\
    agg(count('order_id').alias('order_count')).\
      orderBy(col('order_count').desc())

order_status,order_count
COMPLETE,22899
PENDING_PAYMENT,15030
PROCESSING,8275
PENDING,7610
CLOSED,7556
ON_HOLD,3798
SUSPECTED_FRAUD,1558
CANCELED,1428
PAYMENT_REVIEW,729


In [ ]:
# Mostrar el physical plan
df.explain()

== Physical Plan ==
FileScan csv [order_id#0,order_date#1,order_customer_id#2,order_status#3] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/content/drive/MyDrive/retail_db/orders], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<order_id:int,order_date:date,order_customer_id:int,order_status:string>




# Tuning Columnar format

Una de las estrategias para evitar el sobre proceso cuando se trabajan con archivos planos, es crear particiones de los datos como se muestra a continuación.

In [ ]:
df.\
  write . \
  partitionBy('Year', 'Month'). \
  mode('overwrite'). \
  parquet('dbfs:/FileStore/db')